In [7]:
pwd

'F:\\iNeuron\\End2End\\Simple App'

In [6]:
mlflow.get_artifact_uri()

'mlflow-artifacts:/0/261b7e58a19149bba3e5701a4237e2a9/artifacts'

In [5]:
import mlflow
import os

os.chdir("F:\\iNeuron\\End2End\\Simple App")
# Start a run
with mlflow.start_run():
    # Log metrics, parameters, artifacts, etc.
    mlflow.log_metric("mae", 0.123)
    mlflow.log_param("alpha", 0.5)
    # mlflow.log_artifact("model.pkl")

In [28]:
from urllib.parse import urlparse

tracking_url_type_store = urlparse(mlflow.get_artifact_uri())
tracking_url_type_store

ParseResult(scheme='mlflow-artifacts', netloc='', path='/0/1835e1961b324cea897366c966906701/artifacts', params='', query='', fragment='')

In [27]:
type(tracking_url_type_store)

str

In [10]:
print("MLflow Tracking URI:", mlflow.get_tracking_uri())


MLflow Tracking URI: file:///F:/iNeuron/End2End/Simple%20App/mlruns


In [9]:
from src.get_data import read_params
import argparse
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib
import os

os.chdir("F:\\iNeuron\\End2End\\Simple App")

def log_production_model(config_path):
    config = read_params(config_path)

    mlflow_config = config["mlflow_config"]

    model_name= mlflow_config["registered_model_name"]

    remote_server_uri = mlflow_config['remote_server_uri']
    mlflow.set_registry_uri(remote_server_uri)\

    experiment_id = '415606185164150086'

    # Search for runs in the specified experiment
    runs = mlflow.search_runs(experiment_ids=[experiment_id])

    # Print run IDs
    print("Run IDs:")
    print(runs.columns)


    # runs = mlflow.search_runs(experiment_ids=None)
    # print(runs.columns)
    # print(runs.info())
    # lowest = runs.loc[runs["metrics.mae"].idxmin()]["metrics.mae"]
    # lowest_run_id = runs[runs["metrics.mae"]==lowest]["run_id"][0]

    # client =MlflowClient()
    # for mv in client.search_model_versions(f"name='{model_name}'"):
    #     mv = dict(mv)
    #     pprint(mv, indent=4)

        # if mv["run_id"]== lowest_run_id:
        #     current_version = mv['version']
        #     logged_model = mv["source"]
        #     pprint(mv, indent=4)
        #     client.transition_model_version_stage(
        #         name = model_name,
        #         version = current_version,
        #         stage = 'Production'
        #     )
        # else:
        #     current_version = mv['version']
        #     #logged_model = mv["source"]
        #     #pprint(mv, indent=4)
        #     client.transition_model_version_stage(
        #         name = model_name,
        #         version = current_version,
        #         stage = 'Staging'
        #     )

log_production_model("params.yaml")

Run IDs:
Index(['run_id', 'experiment_id', 'status', 'artifact_uri', 'start_time',
       'end_time', 'metrics.mae', 'metrics.r2', 'metrics.rmse',
       'params.l1_ratio', 'params.alpha', 'tags.mlflow.source.type',
       'tags.mlflow.runName', 'tags.mlflow.source.name',
       'tags.mlflow.source.git.commit', 'tags.mlflow.log-model.history',
       'tags.mlflow.user'],
      dtype='object')


In [1]:
pwd

'f:\\iNeuron\\End2End\\Simple App\\notebooks'

In [39]:
import mlflow

# Replace 'ElasticNet Regression' with the name of your experiment
experiment_name = 'ElasticNet Regression'

# Get the experiment ID by name
experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

AttributeError: 'NoneType' object has no attribute 'experiment_id'

In [38]:
experiment_id

'1'

In [22]:
# Delete the experiment
mlflow.delete_experiment("0")

print(f"Experiment '{experiment_name}' deleted.")

Experiment 'ElasticNet Regression' deleted.


In [27]:
# load the train and test dataset
# train the algorithm
# save the metrics and params

import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
from src.get_data import read_params
import argparse
import joblib
import json

import mlflow

os.chdir("F:\\iNeuron\\End2End\\Simple App")

def eval_metrics(ytest,ypred):
    rmse=np.sqrt(mean_squared_error(ytest,ypred))
    mae=mean_absolute_error(ytest,ypred)
    r2=r2_score(ytest,ypred)
    return rmse,mae,r2

# def train_and_evaluate(config_path):
# args = argparse.ArgumentParser()
# args.add_argument("--config",default="params.yaml")
# parsed_args=args.parse_args()
config_path="F:\\iNeuron\\End2End\\Simple App\\params.yaml"
config = read_params(config_path)

test_data_path = config["split_data"]["test_path"]
train_data_path = config["split_data"]["train_path"]
random_state = config["base"]["random_state"]
model_dir = config["model_dir"]


alpha = config["estimators"]["ElasticNet"]["params"]["alpha"]
l1_ratio = config["estimators"]["ElasticNet"]["params"]["l1_ratio"]


# target = [config["base"]["target_col"]]

# train = pd.read_csv(filepath_or_buffer="F:\\iNeuron\\End2End\\Simple App\\data\\processed\\train_winequality.csv", sep=',',)
# test = pd.read_csv(filepath_or_buffer="F:\\iNeuron\\End2End\\Simple App\\data\\processed\\test_winequality.csv", sep=',',)

# ytrain = train[target]
# ytest = test[target]

# xtrain = train.drop(labels=target,axis=1)
# xtest = test.drop(labels=target,axis=1)


target = [config["base"]["target_col"]]

train = pd.read_csv(train_data_path, sep=',',)
test = pd.read_csv(test_data_path, sep=',',)

ytrain = train[target]
ytest = test[target]

xtrain = train.drop(labels=target,axis=1)
xtest = test.drop(labels=target,axis=1)

################################## MLFLOW BEGINS ##################################

mlflow_config = config["mlflow_config"]
remote_server_uri = mlflow_config["remote_server_uri"]

mlflow.set_tracking_uri(remote_server_uri)
# mlflow.set_experiment(mlflow_config["experiment_name"])
mlflow.set_experiment(mlflow_config["experiment_name"])
with mlflow.start_run():
    model=ElasticNet(alpha=alpha,l1_ratio=l1_ratio, random_state=random_state)
    model.fit(xtrain,ytrain)

    ypred=model.predict(xtest)

    (rmse,mae,r2) = eval_metrics(ytest,ypred)

    mlflow.log_param("alpha",alpha)
    mlflow.log_param("l1_ratio",l1_ratio)

    mlflow.log_metric("rmse", float(rmse)) 
    mlflow.log_metric("mae", float(mae))
    mlflow.log_metric("r2", float(r2)) 

    # The traditional model logging method below is also going to be commented out.
    # Instead, we will use mlflow.
    # os.makedirs(model_dir, exist_ok=True)
    # model_path = os.path.join(model_dir, "model.joblib")

    # joblib.dump(model, model_path)

    tracking_url_type_store = urlparse(mlflow.get_artifact_uri()).scheme
    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(model, 
                                    "model", 
                                    registered_model_name=mlflow_config["registered_model_name"]
                                )
    else:
            mlflow.sklearn.log_model(model, "model")


################################## MLFLOW ENDS ##################################

# if __name__=="__main__":



c:\Users\DELL\anaconda3\envs\venv\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\DELL\anaconda3\envs\venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [28]:
from src.get_data import read_params
import argparse
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib
import os

os.chdir("F:\\iNeuron\\End2End\\Simple App")

config_path="F:\\iNeuron\\End2End\\Simple App\\params.yaml"
config = read_params(config_path)

mlflow_config = config["mlflow_config"]

model_name= mlflow_config["registered_model_name"]

remote_server_uri = mlflow_config['remote_server_uri']
mlflow.set_tracking_uri("http://127.0.0.1:1234")
mlflow.set_registry_uri("http://127.0.0.1:1234")

In [33]:

runs = mlflow.search_runs(experiment_ids=['1'])
lowest = runs["metrics.mae"].sort_values(ascending=True)[0]
lowest_run_id = runs[runs["metrics.mae"]==lowest]["run_id"][0]

In [34]:
client =MlflowClient()

In [36]:
model_name

'ElasticNetWineModel'

In [35]:
client.search_model_versions(f"name='{model_name}'")

[]

In [23]:
for mv in client.search_model_versions(f"name='{model_name}'"):
    mv = dict(mv)
    pprint(mv, indent=4)

    if mv["run_id"]== lowest_run_id:
        current_version = mv['version']
        logged_model = mv["source"]
        pprint(mv, indent=4)
        client.transition_model_version_stage(
            name = model_name,
            version = current_version,
            stage = 'Production'
        )
    else:
        current_version = mv['version']
        logged_model = mv["source"]
        #pprint(mv, indent=4)
        client.transition_model_version_stage(
            name = model_name,
            version = current_version,
            stage = 'Staging'
        )

In [14]:
mv["source"]

NameError: name 'mv' is not defined

In [15]:
logged_model

NameError: name 'logged_model' is not defined

In [47]:
loaded_model = mlflow.pyfunc.load_model(logged_model)
model_path = config["webapp_model_dir"]
joblib.dump(loaded_model,model_path)

MlflowException: The following failures occurred while downloading one or more artifacts from http://127.0.0.1:1234/api/2.0/mlflow-artifacts/artifacts/3/dc2497d0dda84b8a893f5ad2e13a9462/artifacts:
##### File model #####
API request to http://127.0.0.1:1234/api/2.0/mlflow-artifacts/artifacts/3/dc2497d0dda84b8a893f5ad2e13a9462/artifacts/model failed with exception HTTPConnectionPool(host='127.0.0.1', port=1234): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/3/dc2497d0dda84b8a893f5ad2e13a9462/artifacts/model (Caused by ResponseError('too many 500 error responses'))